In [93]:
import os.path
from pathlib import Path
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client
from dask.distributed import progress
from fabric import Connection, Config

In [79]:
client = Client("tcp://128.110.217.91:8786")

/users/prsridha/.local/lib/python3.8/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.7.0  | 1.7.0     | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [95]:
client

<Client: 'tcp://128.110.217.91:8786' processes=2 threads=32, memory=125.38 GiB>

In [96]:
DATA_URL = "metadata.csv"
feature_names = ["image_path", "label"]
dtypes = {'image_path': np.str, 'label': np.int16}
df = dd.read_csv(DATA_URL, names=feature_names, dtype=dtypes)

/tmp/ipykernel_176484/4046133258.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtypes = {'image_path': np.str, 'label': np.int16}


In [97]:
df.head()

,image_path,label
0,data/trainingSet/trainingSet/0/img_28791.jpg,0
1,data/trainingSet/trainingSet/0/img_31920.jpg,0
2,data/trainingSet/trainingSet/0/img_18860.jpg,0
3,data/trainingSet/trainingSet/0/img_9771.jpg,0
4,data/trainingSet/trainingSet/0/img_23377.jpg,0


In [98]:
user = "prsridha"
host = "ms0921.utah.cloudlab.us"
pem_path = "./cloudlab.pem"
connect_kwargs = {"key_filename":pem_path}
conn = Connection(host, user=user, connect_kwargs=connect_kwargs)

def pull(from_path, to_path):
    to_path_dir = "/".join(to_path.split("/")[:-1])
    Path(to_path_dir).mkdir(parents=True, exist_ok=True)
    result = conn.get(from_path, to_path)
    print("Pulled from {0.remote} to {0.local}".format(result))

In [99]:
shuffled_df = df.sample(frac=1)
shuffled_df.head()

,image_path,label
1343,data/trainingSet/trainingSet/0/img_32635.jpg,0
8094,data/trainingSet/trainingSet/1/img_5345.jpg,1
38192,data/trainingSet/trainingSet/9/img_22003.jpg,9
19091,data/trainingSet/trainingSet/4/img_3019.jpg,4
36013,data/trainingSet/trainingSet/8/img_25601.jpg,8


In [100]:
new_df = shuffled_df.repartition(npartitions=2)

In [101]:
def initialize_workers(client):
    """Initialize workers (get worker IPs)"""
    worker_id_ip_dict = {}
    all_worker_details = client.scheduler_info()['workers']
    i = 0
    for ip in all_worker_details:
        # set the mapping between worker ID and worker IP
        worker_id_ip_dict[i] = str(ip)
        i += 1
    return worker_id_ip_dict

In [102]:
workers = initialize_workers(client)

In [103]:
fut1 = client.persist(new_df.partitions[0], workers[0])
fut2 = client.persist(new_df.partitions[1], workers[1])

In [118]:
def transform(path):
    from_path = os.path.join("/users/prsridha", str(path))
    to_path = os.path.join("/users/prsridha", str(path))
    print("PAAATHHH", path)
    pull(from_path, to_path)
    return str(path).split("/")[-1]

In [119]:
# filepaths = new_df.image_path.apply(transform, meta=object)

In [120]:
# filepaths1 = fut1.image_path.apply(transform, meta=object)
# filepaths2 = fut2.image_path.apply(transform, meta=object)
# new_df1 = fut1.assign(filepaths=filepaths1)
# new_df2 = fut2.assign(filepaths=filepaths2)
# new_df3 = dd.concat([new_df1, new_df2])
# new_df3.head()

In [121]:
def transform1(ndf):
    filepaths = ndf.image_path.apply(transform, meta=object)
    ndf.filepaths = filepaths
    return ndf

In [122]:
fut3 = client.submit(transform, fut1, workers=workers[0])
fut4 = client.submit(transform, fut2, workers=workers[1])

In [123]:
progress(fut3)

VBox()

In [124]:
progress(fut4)

VBox()

In [ ]:
conn.close()